**Example:** 

One kmol of propane $\left(C_3H_{8}\right)$ is burned in air under rich conditions in an engine. Assuming rich, incomplete combustion (i.e., 30% of the propane goes to CO in the exhaust), determine (i)the equivalence ratio, and (ii)the combustion temperature.

$C_3H_8 + \phi \left(O_2 + 3.76 N_2\right) \rightarrow n_1CO_2 + n_2H_2O + n_3N_2 + n_4O_2 + n_5CO + n_6H_2$

where, $n_1 = 0.7*3$ and $n_5 = (1-0.7)*3$

(note in this example we're using propane becuase the default chemical mechanisms don't have C8 molecules in them).

In [1]:
# First we import fsolve from scipy
from scipy.optimize import fsolve
from math import exp
import numpy

# Set up the knowns
fuel = 1 #kmol of C3H6
C_fuel = 3
H_fuel = 8

# Note that we were told 70% of the octane converted to CO2 while 30% was converted to CO:
pB = 0.7 #70% burned to CO2

a = pB*(C_fuel+H_fuel/4)
a_s = (C_fuel+H_fuel/4)
phi = a_s/a
print("Phi is calculated to be %1.2f" % phi)

n1 = pB*(C_fuel)
n5 = (1-pB)*C_fuel

# Define our equations to solve
def eqn(vars):
    t, T, K, d1, a1, b1, c1, n2, n6, n3 = vars
    eq1 = t - T/1000
    eq2 = K - exp(2.743 - 1.761/t - 1.611/t**2 + 0.2803/t**3)
    eq3 = d1 - 2*a_s*(1-1/phi)
    eq4 = a1 - (1-K)
    eq5 = b1 - (H_fuel/2 + K*C_fuel - d1*(1-K))
    eq6 = c1 - (-C_fuel*d1*K)
    eq7 = K - n2*n5/(n1*n6)
    eq8 = n2 - (H_fuel/2 - d1 + n5)
    eq9 = n3 - 3.76*a_s/phi
    eq10 = n6 - (d1 - n5)
    return [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10]

# Now call the fsolve function to solve our system of equatins
t, T, K, d1, a1, b1, c1, n2, n6, n3 = fsolve(eqn, (1, 1000, 1, 1, 1, 1, 1, 1, 1, 1))

print("The temperature is  %1.2f K" % T)
#print("n_CO2 = %1.2f" % n1)
#print("n_CO = %1.2f" % n5)

# Calculating mole fractions
ntot = n1+n2+n3+n5+n6 #total moles
x_CO2 = n1/ntot
x_CO = n5/ntot

print("x_CO2 = %1.2f" % x_CO2)
print("x_CO = %1.2f" % x_CO)

Phi is calculated to be 1.43
The temperature is  876.40 K
x_CO2 = 0.10
x_CO = 0.04


%==================

Now let's check our answers with cantera:

In [4]:
# First you need to import the requried toolboxes
import cantera as ct

# Let's create the gas object
gas1 = ct.Solution('gri30.yaml')
gas1.set_equivalence_ratio(phi=phi, fuel='C3H8', oxidizer={'O2':1.0, 'N2':3.76}) #note we pulled "phi" from above
gas1.TP = T, 101325     #let's pull the temp from up above
#check that we've set the equivalence ratio correctly
g1eqr = gas1.equivalence_ratio()
print("The equivalence ratio prior to equilb calc is: %1.2f" % g1eqr)

# Solve the const enthaply equation
gas1.equilibrate('TP')

# Let's search for specific products
mf_co2 = gas1['CO2'].X
print("The mole fraction of CO2 is %1.4f " % mf_co2)
mf_co = gas1['CO'].X
print("The mole fraction of CO is %1.4f " % mf_co)

gas1()

The equivalence ratio prior to equilb calc is: 1.43
The mole fraction of CO2 is 0.1049 
The mole fraction of CO is 0.0433 

  gri30:

       temperature   876.4 K
          pressure   1.0132e+05 Pa
           density   0.36256 kg/m^3
  mean mol. weight   26.074 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.9285e+06       -5.0282e+07  J
   internal energy       -2.2079e+06       -5.7569e+07  J
           entropy              8876        2.3143e+05  J/K
    Gibbs function       -9.7075e+06       -2.5311e+08  J
 heat capacity c_p            1330.9             34701  J/K
 heat capacity c_v              1012             26386  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2         0.0079137           0.10235           -19.467
                 H  